In [1]:
import pandas as pd
import numpy as np
import os
import csv
import pickle

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Num GPUs Available:  1
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1



# Quetsions to ask:

- What does 1d vs 2 architecture actually mean?
- What model does he think is the best?
- Visualization? Attribution map? 1d version of saliecy

- Sliding window approach -> predict on segmets and take element wise maximum to produce single prediction for whole sample (window wize 2.5 second)
- element wise maximum for test set? or is that included in the training?
- preprocessing


- map signal into a frequency domain
decompose waveform into frequency componenets, comination of sin waves with differenct frequencies -> spectrogram time vs frequency vs color
makes it a 2d problem
spectrogram is already in scipy



# Test Train Split

In [2]:
# Load data from cleaned pickles
database = pd.read_pickle('data/database.pkl')
ecg_data = pd.read_pickle('data/ecg_data.pkl')

X = np.array(ecg_data)
Y = np.array(database['class'])

In [4]:
# Split data from folds provided by ptb xl
val_fold  = [8,9]
test_fold = [10] 

train_idx = np.where(np.isin(database['strat_fold'], [1,2,3], invert=True))
# train_idx = np.where(np.isin(database['strat_fold'], val_fold+test_fold, invert=True))
val_idx = np.where(np.isin(database['strat_fold'], val_fold))
test_idx = np.where(np.isin(database['strat_fold'], test_fold))

X_train = list(X[train_idx])
X_val   = list(X[val_idx])
X_test  = list(X[test_idx])
Y_train = list(Y[train_idx])
Y_val   = list(Y[val_idx])
Y_test  = list(Y[test_idx])

In [6]:
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
# val_dataset   = tf.data.Dataset.from_tensor_slices((X_val, Y_val))
# test_dataset  = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(1)

In [105]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = len(X_train)

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(1)

TypeError: The dataset length is unknown.

In [164]:
num_classes = len(np.unique(Y))
input_shape = X[0].shape

print('num_classes =', num_classes)
print('input_shape =', input_shape)

num_classes = 23
input_shape = (12, 1000)


## Sliding Window

In [176]:
def sliding_window(data_label_tuple, sequence_length=250, sequence_stride=50):
    '''
    Creates a nested tuple ((array,label),(array,label)) length sequence_length
    and stride sequence_stride from original array in data_label_tuple.
    
    ***Shape must match***
    
    INPUTS:
        data_label_tuple: tuple of length 2, (original_array, label)
        sequence length: length of resulting sequences
        sequence stride: stride between initial index of sequences
        
    outputs:
        nested tuple of form ((array, label),(array,label))
    array data
    '''
    
    sequence = data_label_tuple[0]
    label = data_label_tuple[1]
    n_sequences = int((sequence.shape[1]-sequence_length)/sequence_stride)
    
    start_idx = 0
    seq_labels = ()
    for i in range(n_sequences):
        seq = sequence[:,start_idx:start_idx+sequence_length]
        start_idx += sequence_stride
        seq_labels += ((seq,label),)
    
    return seq_labels


for elem in train_dataset.take(1).as_numpy_iterator():
    input_tuple = elem
sequence_length = 250
sequence_stride = 50

temp = sliding_window(input_tuple, sequence_length, sequence_stride)
temp[0][0].shape

(12, 250)

In [7]:
def sliding_window(array, label, sequence_length=250, sequence_stride=50):
    '''
    Creates a tuple of arrays of length sequence_length and stride sequence_stride. 
    
    INPUTS:
        array: numpy array of 1d or 2d sequential data
        sequence length: length of resulting sequences
        sequence stride: stride between initial index of sequences
        
    OUTPUTS:
        tuple of arrays
    '''
    n_sequences = int((array.shape[1]-sequence_length)/sequence_stride)
    labels = [label]*n_sequences
    
    start_idx = 0
    seqs = []
    for i in range(n_sequences):
        seq = array[:,start_idx:start_idx+sequence_length]
        start_idx += sequence_stride
        seqs.append(seq)
    
    return seqs, labels

def apply_sliding_window(X, Y, sequence_length=250, sequence_stride=50):
    '''
    Applies sliding window to a list of arrays
    '''
    X_windows = []
    Y_windows = []
    for i in range(len(X)):
        seqs, labels = sliding_window(X[i], Y[i], sequence_length, sequence_stride)
        X_windows += seqs 
        Y_windows += labels
    return X_windows, Y_windows

In [8]:
sequence_length = 250
sequence_stride = 50

X_train_windows, Y_train_windows = apply_sliding_window(X_train, Y_train, sequence_length, sequence_stride)
X_val_windows, Y_val_windows = apply_sliding_window(X_val, Y_val, sequence_length, sequence_stride)
X_test_windows, Y_test_windows = apply_sliding_window(X_test, Y_test, sequence_length, sequence_stride)

In [68]:
# Serialize ecg data
record_file = 'X_train.tfrecord'
with tf.io.TFRecordWriter(record_file) as writer:
    for i in range(len(X_train_windows)):
        serialized_ecg = tf.io.serialize_tensor(X_train_windows[i])
        writer.write(serialized_ecg.numpy())

In [9]:
# Read from file
parse_tensor = lambda x: tf.io.parse_tensor(x, tf.double)
X_train = (tf.data.TFRecordDataset('X_train.tfrecord').map(parse_tensor))
X_train = X_train.map(lambda x: tf.reshape(x, [12,250]))

In [86]:
# Serialize labels
record_file = 'Y_train.tfrecord'
with tf.io.TFRecordWriter(record_file) as writer:
    for i in range(len(Y_train_windows)):
        serialized_label = tf.io.serialize_tensor(Y_train_windows[i])
        writer.write(serialized_label.numpy())

In [30]:
# Read from file
parse_labels = lambda x: tf.io.parse_tensor(x, tf.int64)
Y_train = (tf.data.TFRecordDataset('Y_train.tfrecord').map(parse_labels))
Y_train = Y_train.map(lambda x: tf.reshape(x, []))

In [38]:
BATCH_SIZE = 256
SHUFFLE_BUFFER_SIZE = len(X_train_windows)

train_ds = tf.data.Dataset.zip((X_train, Y_train))
train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
# print(train_ds)

In [ ]:
# for data, label in train_ds.take(1).as_numpy_iterator():
#     x = data 
#     print(x)
#     y = label
#     print(y)

In [34]:
num_classes = len(np.unique(Y))
input_shape = X_train_windows[0].shape

print('num_classes =', num_classes)
print('input_shape =', input_shape)

num_classes = 23
input_shape = (12, 250)


In [ ]:
def map_example(height, width, image_raw, label):
    image_data = tf.io.decode_raw(image_raw, tf.uint8)
    image_data = tf.reshape(image_data, [1, height, width])
    return image_data, label

def make_dataset(partition):
    # dataset = dataset.shuffle(buffer_size=FLAGS.shuffle_buffer_size)
    dataset = dataset.map(decode_example)
    dataset = dataset.map(
        lambda x: map_example(x['height'], x['width'], x['image_raw'], x['label']))
    # dataset = dataset.batch(batch_size=FLAGS.batch_size)
    return dataset

In [145]:
test_windows = [X_test_windows,Y_test_windows]

tf records, save everythign into harddrive first
dump everythign into tf record file, each sample becomes a tf record (sits on hard drive)
create a tf record dataset, tf has pointers to the HDD to pull up data

In [95]:
# save file as pickle
with open('data/test.pkl', 'wb') as f:
    pickle.dump(X_test_windows, f)

In [96]:
# load pickle
with open('data/test.pkl', 'rb') as f:
    mynewlist = pickle.load(f)

In [137]:
def save_to_multiple_pickles(data, name_prefix, n_parts=10):
    data_dir = 'data'
    path_format = os.path.join(data_dir, "{}_{:02d}.pkl")

    filepaths = []
    m = len(data)
    temp = list(enumerate(np.array_split(np.arange(m), n_parts)))
    
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_pkl = path_format.format(name_prefix, file_idx)
        filepaths.append(part_pkl)
        with open(part_pkl, 'wb') as f:
            pickle.dump([data[i] for i in row_indices], f)

    return filepaths

In [148]:
X_train_filepaths = save_to_multiple_pickles(X_train_windows, 'X_train', n_parts=10)
Y_train_filepaths = save_to_multiple_pickles(Y_train_windows, 'Y_train', n_parts=10)
X_val_filepaths = save_to_multiple_pickles(X_val_windows, 'X_val', n_parts=4)
Y_val_filepaths = save_to_multiple_pickles(Y_val_windows, 'Y_val', n_parts=4)
X_test_filepaths = save_to_multiple_pickles(X_test_windows, 'X_test', n_parts=2)
Y_test_filepaths = save_to_multiple_pickles(Y_test_windows, 'Y_test', n_parts=2)

In [152]:
# load pickle
with open(X_train_filepaths[0], 'rb') as f:
    mynewlist = pickle.load(f)

### Input pipeline

In [161]:
filepath_dataset = tf.data.Dataset.list_files(X_train_filepaths, seed=22)

In [162]:
for filepath in filepath_dataset:
    print(filepath)

tf.Tensor(b'data\\X_train_06.pkl', shape=(), dtype=string)
tf.Tensor(b'data\\X_train_04.pkl', shape=(), dtype=string)
tf.Tensor(b'data\\X_train_01.pkl', shape=(), dtype=string)
tf.Tensor(b'data\\X_train_05.pkl', shape=(), dtype=string)
tf.Tensor(b'data\\X_train_08.pkl', shape=(), dtype=string)
tf.Tensor(b'data\\X_train_00.pkl', shape=(), dtype=string)
tf.Tensor(b'data\\X_train_09.pkl', shape=(), dtype=string)
tf.Tensor(b'data\\X_train_07.pkl', shape=(), dtype=string)
tf.Tensor(b'data\\X_train_03.pkl', shape=(), dtype=string)
tf.Tensor(b'data\\X_train_02.pkl', shape=(), dtype=string)


In [27]:
train_dataset_windows = tf.data.Dataset.from_tensor_slices((X_train_windows, Y_train_windows))
val_dataset   = tf.data.Dataset.from_tensor_slices((X_val_windows, Y_val_windows))
test_dataset  = tf.data.Dataset.from_tensor_slices((X_test_windows, Y_test_windows))

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = len(X_train)

train_dataset_windows = train_dataset_windows.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)#.window(size=250, shift=50)
val_dataset_windows = val_dataset_windows.batch(BATCH_SIZE)#.window(size=250, shift=50)
test_dataset_windows = test_dataset_windows.batch(1)#.window(size=250, shift=50)

input_shape = (12, 250)

In [53]:
window_ds = train_dataset.map(lambda x, y: (sliding_window(x), y))
for x, y in window_ds.take(1):
    print(x)
    print(y)
    
for x, y in train_dataset.take(1):
    print(x)

(<tf.Tensor: shape=(12, 250), dtype=float64, numpy=
array([[-0.119, -0.116, -0.12 , ..., -0.064, -0.073, -0.093],
       [-0.055, -0.051, -0.044, ...,  0.009, -0.007, -0.024],
       [ 0.064,  0.065,  0.076, ...,  0.073,  0.065,  0.068],
       ...,
       [-0.026, -0.031, -0.028, ..., -0.022, -0.025, -0.03 ],
       [-0.039, -0.034, -0.029, ..., -0.019, -0.025, -0.035],
       [-0.079, -0.074, -0.069, ...,  0.017,  0.015,  0.011]])>, <tf.Tensor: shape=(12, 250), dtype=float64, numpy=
array([[ 0.021,  0.011, -0.009, ..., -0.011, -0.033,  0.204],
       [ 0.151,  0.137,  0.117, ..., -0.026,  0.014,  0.227],
       [ 0.13 ,  0.126,  0.127, ..., -0.014,  0.046,  0.023],
       ...,
       [ 0.176,  0.171,  0.153, ..., -0.037,  0.023,  0.356],
       [ 0.177,  0.16 ,  0.149, ..., -0.044, -0.022,  0.248],
       [ 0.114,  0.106,  0.103, ..., -0.017, -0.006,  0.171]])>, <tf.Tensor: shape=(12, 250), dtype=float64, numpy=
array([[-0.051, -0.072, -0.098, ..., -0.058, -0.033,  0.021],
       [ 0

In [195]:
# train_dataset.take(1)
# for ecg, label in ds.as_numpy_iterator():
#     print(ecg.shape)

# for elem in train_dataset.take(1):
#     print(elem.map(sliding_window))

temp_ds = train_dataset.take(2).map(lambda ecg, label: sliding_window(ecg, label))
# for elem in temp_ds:
#     print(len(elem))
    
# print(type(elem))
# print(len(elem))
# print(elem[0])
# print(elem[1])
# print(x)
    
# train_dataset.map(sliding_window)

ValueError: in user code:

    File "C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_22972/834844556.py", line 8, in None  *
        lambda ecg, label: sliding_window(ecg, label)
    File "C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_22972/1376489108.py", line 24, in sliding_window  *
        for i in range(n_sequences):

    ValueError: Shape must be rank 0 but is rank 1
    	 for 'limit' for '{{node range}} = Range[Tidx=DT_INT32](range/start, Maximum, range/delta)' with input shapes: [], [?], [].


In [65]:
total_length    = X_train[0].shape[1]
sequence_length = 250
sequence_stride = 50

temp_ds = keras.utils.timeseries_dataset_from_array(data=X_test,
                                                   targets=np.repeat(Y_test,(total_length-sequence_length)/sequence_stride),
                                                   sequence_length=sequence_length,
                                                   sequence_stride=sequence_stride,
                                                   sampling_rate=1,
                                                   batch_size=None)

In [82]:
window_ds = train_dataset.take(1).window(size=250, shift=50)
window = next(iter(window_ds))
for elem in window[0].as_numpy_iterator():
    x = elem

In [83]:
train_dataset_windows = train_dataset.map(lambda ecg, label: sliding_window((ecg, label)))
val_dataset_windows = val_dataset.map(lambda ecg, label: sliding_window((ecg, label)))

AttributeError: in user code:

    File "C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_30348/3721291042.py", line 1, in None  *
        lambda ecg, label: sliding_window((ecg, label))
    File "C:\Users\Public\Documents\Wondershare\CreatorTemp/ipykernel_30348/644226537.py", line 15, in sliding_window  *
        n_sequences = int((array.shape[1]-sequence_length)/sequence_stride)

    AttributeError: 'tuple' object has no attribute 'shape'


## Recrop

In [81]:
import random

def random_recrop(array, sequence_length=250):

    array_length = array.shape[1]
    start_idx = random.randint(0, array_length-sequence_length-1)
    seq = array[:,start_idx:start_idx+sequence_length]

    return seq

array = x #X_train[0]
sequence_length = 250
sequence_stride = 50

random_recrop(array, sequence_length)

array([[ 0.043,  0.025, -0.01 , ...,  0.091,  0.494,  0.572],
       [ 0.11 ,  0.072,  0.043, ...,  0.13 ,  0.338,  0.317],
       [ 0.067,  0.047,  0.053, ...,  0.039, -0.156, -0.254],
       ...,
       [ 0.107,  0.089,  0.044, ...,  0.179,  0.495,  0.231],
       [ 0.103,  0.077,  0.027, ...,  0.104,  0.415,  0.413],
       [ 0.124,  0.088,  0.062, ..., -0.014,  0.23 ,  0.369]])

In [64]:
for x, y in train_dataset.take(1).as_numpy_iterator():
    temp = x

In [97]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = len(X_train)

train_dataset_recrop = train_dataset.map(lambda x, y: (random_recrop(x), y)).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset_recrop = val_dataset.map(lambda x, y: (random_recrop(x), y)).batch(BATCH_SIZE)

input_shape = (12, 250)

# Model 1: Basic CNN

In [39]:
input_shape

(12, 250)

In [40]:
model = keras.models.Sequential([
    
    keras.layers.Conv1D(filters=64, kernel_size=3, padding="same", input_shape=input_shape),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),

    keras.layers.Conv1D(filters=64, kernel_size=3, padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),

    keras.layers.Conv1D(filters=64, kernel_size=3, padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.ReLU(),

    keras.layers.GlobalAveragePooling1D(),

    keras.layers.Dense(num_classes, activation="softmax")
])

model.build(input_shape)

# model.summary()

In [41]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [42]:
epochs = 15

callbacks = [keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss"),
             keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001),
             keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),]


history = model.fit(train_ds,
#                     validation_data=val_dataset_recrop,
                    epochs=epochs,
#                     batch_size=batch_size,
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/15
    869/Unknown - 1247s 1s/step - loss: 1.7514 - accuracy: 0.4934WARNING:tensorflow:Can save best model only with val_loss available, skipping.
869/869 [==============================] - 1247s 1s/step - loss: 1.7514 - accuracy: 0.4934 - lr: 0.0010
Epoch 2/15


KeyboardInterrupt: 

In [13]:
epochs = 15

callbacks = [keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss"),
             keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001),
             keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),]


history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=epochs,
#                     batch_size=batch_size,
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/15
463/463 [==============================] - 18s 27ms/step - loss: 1.7386 - accuracy: 0.5145 - val_loss: 1.6803 - val_accuracy: 0.5317 - lr: 0.0010
Epoch 2/15
463/463 [==============================] - 12s 26ms/step - loss: 1.4288 - accuracy: 0.5831 - val_loss: 1.4921 - val_accuracy: 0.5482 - lr: 0.0010
Epoch 3/15
463/463 [==============================] - 12s 25ms/step - loss: 1.3104 - accuracy: 0.6097 - val_loss: 1.3365 - val_accuracy: 0.6112 - lr: 0.0010
Epoch 4/15
463/463 [==============================] - 12s 26ms/step - loss: 1.2401 - accuracy: 0.6288 - val_loss: 1.3336 - val_accuracy: 0.5914 - lr: 0.0010
Epoch 5/15
463/463 [==============================] - 12s 25ms/step - loss: 1.1879 - accuracy: 0.6417 - val_loss: 1.2776 - val_accuracy: 0.6211 - lr: 0.0010
Epoch 6/15
463/463 [==============================] - 12s 26ms/step - loss: 1.1568 - accuracy: 0.6505 - val_loss: 1.2451 - val_accuracy: 0.6206 - lr: 0.0010
Epoch 7/15
463/463 [==============================] - 12s 

In [100]:
epochs = 15

callbacks = [keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss"),
             keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001),
             keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),]


history = model.fit(train_dataset_recrop,
                    validation_data=val_dataset_recrop,
                    epochs=epochs,
#                     batch_size=batch_size,
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/15
463/463 [==============================] - 12s 21ms/step - loss: 2.0273 - accuracy: 0.4395 - val_loss: 1.9192 - val_accuracy: 0.4551 - lr: 0.0010
Epoch 2/15
463/463 [==============================] - 11s 22ms/step - loss: 1.7591 - accuracy: 0.4868 - val_loss: 1.8674 - val_accuracy: 0.4624 - lr: 0.0010
Epoch 3/15
463/463 [==============================] - 10s 21ms/step - loss: 1.6453 - accuracy: 0.5135 - val_loss: 1.8349 - val_accuracy: 0.4827 - lr: 0.0010
Epoch 4/15
463/463 [==============================] - 10s 22ms/step - loss: 1.5395 - accuracy: 0.5448 - val_loss: 1.8403 - val_accuracy: 0.4735 - lr: 0.0010
Epoch 5/15
463/463 [==============================] - 9s 19ms/step - loss: 1.4533 - accuracy: 0.5651 - val_loss: 1.8895 - val_accuracy: 0.4633 - lr: 0.0010
Epoch 6/15
463/463 [==============================] - 9s 19ms/step - loss: 1.3665 - accuracy: 0.5900 - val_loss: 1.9099 - val_accuracy: 0.4589 - lr: 0.0010
Epoch 7/15
463/463 [==============================] - 9s 19m

In [14]:
# model = keras.models.load_model('best_model.h5')
model.evaluate(test_dataset)

2138/2138 [==============================] - 11s 5ms/step - loss: 1.2009 - accuracy: 0.6408


[1.2008675336837769, 0.6407857537269592]

# Model 2: Resnet

when tuning start with learning rate->mini_batch_size -> momentum-> #hidden_units -> # learning_rate_decay -> #layers 

In [36]:
def resnet1d(input_shape, nb_classes):
    
    n_feature_maps = 64

    input_layer = keras.layers.Input(input_shape)

    # BLOCK 1
    conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(input_layer)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum
    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_1 = keras.layers.add([shortcut_y, conv_z])
    output_block_1 = keras.layers.Activation('relu')(output_block_1)
    # insert droupout layer

    
    # BLOCK 2
    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(output_block_1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum
    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_2 = keras.layers.add([shortcut_y, conv_z])
    output_block_2 = keras.layers.Activation('relu')(output_block_2)
    # insert droupout layer

    
    # BLOCK 3
    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(output_block_2)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # no need to expand channels because they are equal
    shortcut_y = keras.layers.BatchNormalization()(output_block_2)

    output_block_3 = keras.layers.add([shortcut_y, conv_z])
    output_block_3 = keras.layers.Activation('relu')(output_block_3)
    
    # insert droupout layer

    
    # FINAL
    gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)
    
    # insert droupout layer

    output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


    return model

to help with overfitting (a generalizability problem):
- windows, increases data size
- add dropout layers
- augmentations: add random noise


try standardizing? won't necessarily help with overfitting

In [37]:
model = resnet1d(input_shape = input_shape,
                 nb_classes = num_classes)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 12, 1000)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 12, 64)       192064      ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 12, 64)      256         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 12, 64)       0           ['batch_normalization[0][0]']

In [26]:
epochs = 15

callbacks = [keras.callbacks.ModelCheckpoint("resnet_model.h5", save_best_only=True, monitor="val_loss"),
             keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001),
             keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),]

history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=epochs,
                  # batch_size=batch_size,
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/15
463/463 [==============================] - 27s 38ms/step - loss: 2.0161 - accuracy: 0.4417 - val_loss: 2.0684 - val_accuracy: 0.4530 - lr: 0.0010
Epoch 2/15
463/463 [==============================] - 17s 36ms/step - loss: 1.7190 - accuracy: 0.4960 - val_loss: 1.9722 - val_accuracy: 0.4589 - lr: 0.0010
Epoch 3/15
463/463 [==============================] - 17s 36ms/step - loss: 1.4807 - accuracy: 0.5595 - val_loss: 1.9561 - val_accuracy: 0.4548 - lr: 0.0010
Epoch 4/15
463/463 [==============================] - 16s 36ms/step - loss: 1.2079 - accuracy: 0.6375 - val_loss: 2.1000 - val_accuracy: 0.4240 - lr: 0.0010
Epoch 5/15
463/463 [==============================] - 16s 35ms/step - loss: 0.9429 - accuracy: 0.7130 - val_loss: 2.3132 - val_accuracy: 0.4079 - lr: 0.0010
Epoch 6/15
463/463 [==============================] - 17s 36ms/step - loss: 0.7176 - accuracy: 0.7824 - val_loss: 2.7267 - val_accuracy: 0.3954 - lr: 0.0010
Epoch 7/15
463/463 [==============================] - 16s 

In [103]:
# Recropped model

epochs = 15

callbacks = [keras.callbacks.ModelCheckpoint("resnet_model.h5", save_best_only=True, monitor="val_loss"),
             keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001),
             keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),]

history = model.fit(train_dataset_recrop,
                    validation_data=val_dataset_recrop,
                    epochs=epochs,
#                     batch_size=batch_size,
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/15
463/463 [==============================] - 23s 41ms/step - loss: 1.9944 - accuracy: 0.4418 - val_loss: 1.9357 - val_accuracy: 0.4572 - lr: 0.0010
Epoch 2/15
463/463 [==============================] - 19s 40ms/step - loss: 1.6913 - accuracy: 0.5018 - val_loss: 1.8578 - val_accuracy: 0.4777 - lr: 0.0010
Epoch 3/15
463/463 [==============================] - 19s 41ms/step - loss: 1.5238 - accuracy: 0.5421 - val_loss: 1.7408 - val_accuracy: 0.4808 - lr: 0.0010
Epoch 4/15
463/463 [==============================] - 19s 40ms/step - loss: 1.3623 - accuracy: 0.5883 - val_loss: 1.7952 - val_accuracy: 0.4968 - lr: 0.0010
Epoch 5/15
463/463 [==============================] - 18s 39ms/step - loss: 1.2238 - accuracy: 0.6283 - val_loss: 1.8533 - val_accuracy: 0.4617 - lr: 0.0010
Epoch 6/15
463/463 [==============================] - 18s 39ms/step - loss: 1.0891 - accuracy: 0.6655 - val_loss: 1.8807 - val_accuracy: 0.4905 - lr: 0.0010
Epoch 7/15
463/463 [==============================] - 20s 